In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2
import numpy as np
from numpy.linalg import norm

# Load pre-trained VGG16 model with ImageNet weights
model = VGG16(weights='imagenet', include_top=False)

# Summary of the model architecture
model.summary()

# Load and preprocess the image from a local file path
def preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image from the local file path
    if img is None:
        raise Exception(f"Unable to load image at path: {image_path}")
    
    img = cv2.resize(img, (224, 224))  # Resize to 224x224
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Normalize for the CNN
    return img

# Function to describe the difference based on the distance
def describe_difference(distance):
    if distance < 100:
        return "The images are very similar, with almost no noticeable differences."
    elif 100 <= distance < 500:
        return "The images have some noticeable differences, but they are still quite similar."
    elif 500 <= distance < 1000:
        return "The images have several differences, and they appear moderately distinct from one another."
    else:
        return "The images are very different, with significant changes in content or appearance."



def cnn_process(image_path1, image_path2) :
    
  # Preprocess the images
  image1 = preprocess_image(image_path1)
  image2 = preprocess_image(image_path2)

  # Extract features from the last convolutional layer
  features1 = model.predict(image1)
  features2 = model.predict(image2)

  # Flatten the feature vectors
  features1 = features1.flatten()
  features2 = features2.flatten()

  # Calculate Euclidean distance
  distance = norm(features1 - features2)
  print(f'Euclidean Distance between the images: {distance}')

  # Get a sentence describing the difference
  description = describe_difference(distance)
  print(description)





Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
cnn_process('lectureex1.png', 'lectureex2.png')

Exception: Unable to load image at path: lectureex1.png

In [5]:
cnn_process('lectureex1.png', 'bunny.png')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Euclidean Distance between the images: 1617.4779052734375
The images are very different, with significant changes in content or appearance.


In [7]:
cnn_process('lectureex1.png', 'lectureex3.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Euclidean Distance between the images: 988.3521118164062
The images have several differences, and they appear moderately distinct from one another.


In [4]:
import cv2
import os

def extract_and_save_scenes(video_path, output_dir):
    """
    Extracts one frame per second from the provided video, saves them as image files locally,
    and returns scenes as a dictionary.

    Parameters:
    video_path (string): Path of the video to be processed.
    output_dir (string): Directory where the extracted frames will be saved.

    Returns:
    list of dict: Each dictionary corresponds to a scene, containing:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    """
    
    if not os.path.exists(video_path):
        print(f"{video_path}: File not found.")
        return []

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps <= 0:
        print(f"Error: Could not retrieve FPS from {video_path}")
        return []

    scenes = []
    index = 0
    success, frame = cap.read()
    count = 0

    while success:
        # Capture frame every second (approximately)
        if count % int(fps) == 0:
            # Define the file name for the saved image
            img_file = os.path.join(output_dir, f"scene_{index}.jpg")
            
            # Save the frame as an image file
            cv2.imwrite(img_file, frame)
            
            # Append scene info to the list
            scenes.append({
                "index": index,
                "img_file": img_file
            })
            
            index += 1
        
        success, frame = cap.read()
        count += 1

    cap.release()
    
    return scenes

In [5]:
import os

def compare_scenes_in_reverse(scenes, cnn_process):
    """
    Compares all scenes in reverse order using the cnn_process function.

    Parameters:
    scenes (list of dict): List of dictionaries where each dict contains:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    cnn_process (function): Function to compare two images.

    Returns:
    None
    """

    # Compare images in reverse order
    for i in range(len(scenes) - 1, 0, -1):
        image_path1 = scenes[i]["img_file"]
        image_path2 = scenes[i - 1]["img_file"]
        
        print(f"Comparing Scene {scenes[i]['index']} and Scene {scenes[i-1]['index']}:")
        cnn_process(image_path1, image_path2)
        print("-" * 50)  # Separator for readability

# Example Usage:
# Assuming 'scenes' is the list returned by extract_and_save_scenes() function
# and cnn_process() is the function to compare two images.
# scenes = extract_and_save_scenes("path_to_video.mp4", "output_directory")
# compare_scenes_in_reverse(scenes, cnn_process)


In [7]:
video_path = "test0_toy_lecture.mp4"
output_dir = "out_directory0"
scenes = extract_and_save_scenes(video_path, output_dir)
print(f"Extracted and saved {len(scenes)} scenes.")
compare_scenes_in_reverse(scenes, cnn_process)

Extracted and saved 38 scenes.
Comparing Scene 37 and Scene 36:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Euclidean Distance between the images: 99.49703216552734
The images are very similar, with almost no noticeable differences.
--------------------------------------------------
Comparing Scene 36 and Scene 35:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Euclidean Distance between the images: 12.08437728881836
The images are very similar, with almost no noticeable differences.
--------------------------------------------------
Comparing Scene 35 and Scene 34:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Euclidean Distance between the images: 98.90432739257812
The images are very similar, with almost no noticeable differences.
--------------------------------------------------
Comparing Scene 34 and Scene 33:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
